In [12]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import unicodedata
import os
import webbrowser
import html5lib
from openpyxl import workbook
from datetime import datetime
import requests

In [23]:
path = '../Master_tables_MFIC_Investment.xlsx'
dataframes = pd.read_excel(path, sheet_name=None)


dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")


process_tables = {}
process_tables_shape = {}
if not os.path.exists('../PT_csv_file'):
    os.makedirs('../PT_csv_file')


def run_process_function(dataframes, process_tables, process_tables_shape):
    path = '../process_tables_GBDC_Investment.xlsx'
    writer = pd.ExcelWriter(path=path, engine='openpyxl')
    for dataframe in dataframes:
        print(dataframe)
        processed_table = process_table_function(dataframes[dataframe])
        process_tables[dataframe] = processed_table
        process_tables_shape[dataframe] = processed_table.shape
        processed_table.to_excel(
            writer, sheet_name=dataframe.replace(',', ''), index=False)
        processed_table.to_csv(
            '../PT_csv_file/'+dataframe.replace(',', '')+'.csv')
        writer.book.save(path)
    writer.close()


def shape(count, df):
    print(f"{count} : shape : {df.shape}")
    count += 1
    return count


def dropna_col_row(df):
    df = df.dropna(how='all', axis=0).reset_index(drop=True)
    df = df.dropna(how='all', axis=1).reset_index(drop=True)
    return df


def drop_if_contain(pattern, df):
    matching_rows = df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    df = df[~matching_rows]
    return df


def rename_columns(df):
    num_cols = df.shape[1]
    data_col_mapper = dict(
        zip(df.columns.to_list(), [i for i in range(0, num_cols)]))
    df = df.rename(columns=data_col_mapper)
    return df

DataFrame name: September 30 2023 : (983, 45)
DataFrame name: June 30 2023 : (1043, 45)
DataFrame name: March 31 2023 : (986, 45)
DataFrame name: December 31 2022 : (890, 45)
DataFrame name: September 30 2022 : (950, 45)
DataFrame name: June 30 2022 : (961, 50)
DataFrame name: March 31 2022 : (915, 50)
DataFrame name: December 31 2021 : (935, 50)
DataFrame name: September 30 2021 : (982, 50)
DataFrame name: June 30 2021 : (1050, 50)
DataFrame name: March 31 2021 : (1006, 50)
DataFrame name: December 31 2020 : (1023, 50)
DataFrame name: September 30 2020 : (1059, 50)
DataFrame name: June 30 2020 : (1109, 50)
DataFrame name: March 31 2020 : (1114, 50)
DataFrame name: December 31 2019 : (872, 27)
DataFrame name: September 30 2019 : (639, 21)
DataFrame name: June 30 2019 : (582, 21)
DataFrame name: March 31 2019 : (492, 21)
DataFrame name: December 31 2018 : (422, 21)
DataFrame name: September 30 2018 : (403, 26)
DataFrame name: June 30 2018 : (369, 21)
DataFrame name: March 31 2018 : (367

In [14]:
def process_table_function(soi_table_df):
    count = 1
    count = shape(count, soi_table_df)
    soi_table_df = soi_table_df.replace(
        r'^\s*\$\s*$', '', regex=True).replace(r'\n', '', regex=True)
    soi_table_df = soi_table_df.replace('-', '0')
    soi_table_df = dropna_col_row(soi_table_df)
    soi_table_df = soi_table_df.apply(
        lambda x: x.strip() if isinstance(x, str) else x)
    count = shape(count, soi_table_df)

    # drops all the extra top row
    pattern = r'Industry'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)
    count = shape(count, soi_table_df)

    return soi_table_df


run_process_function(dataframes=dataframes, process_tables=process_tables,
                     process_tables_shape=process_tables_shape)

September_30_2023
1 : shape : (983, 45)
2 : shape : (983, 45)
June_30_2023
1 : shape : (1043, 45)
2 : shape : (1043, 45)
March_31_2023
1 : shape : (986, 45)
2 : shape : (986, 45)
December_31_2022
1 : shape : (890, 45)
2 : shape : (890, 45)
September_30_2022
1 : shape : (950, 45)
2 : shape : (950, 45)
June_30_2022
1 : shape : (961, 50)
2 : shape : (961, 50)
March_31_2022
1 : shape : (915, 50)
2 : shape : (915, 50)
December_31_2021
1 : shape : (935, 50)
2 : shape : (935, 50)
September_30_2021
1 : shape : (982, 50)
2 : shape : (982, 50)
June_30_2021
1 : shape : (1050, 50)
2 : shape : (1050, 50)
March_31_2021
1 : shape : (1006, 50)
2 : shape : (1006, 50)
December_31_2020
1 : shape : (1023, 50)
2 : shape : (1023, 50)
September_30_2020
1 : shape : (1059, 50)
2 : shape : (1059, 50)
June_30_2020
1 : shape : (1109, 50)
2 : shape : (1109, 50)
March_31_2020
1 : shape : (1114, 50)
2 : shape : (1114, 50)
December_31_2019
1 : shape : (872, 27)
2 : shape : (872, 27)
September_30_2019
1 : shape : (639

In [16]:
process_tables['December_31_2022'].to_csv("test.csv")

In [21]:
process_tables['December_31_2022'].dropna(axis=0, inplace=True)

In [22]:
process_tables['December_31_2022'].shape

(890, 45)